In [1]:
import json
import pandas as pd       
import os
import nltk
from nltk.corpus import stopwords


# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words


# Load the model that we created in Part 2 
from gensim.models import Word2Vec
import os


# Open Trining File
os.chdir("/temp/DocumentCorpus/ECHR/echr_dataset-master/echr_dataset-master/document_crawler/IN/")
train = pd.read_csv("TRAIN_classifiedFULL_NOADMISS.tsv", header=0, delimiter="^", quoting=3)
test  = pd.read_csv("TEST_NOclassifiedFULL_NOADMISS.tsv", header=0, delimiter="^", quoting=3)

os.chdir("/temp/DocumentCorpus/ECHR/echr_dataset-master/echr_dataset-master/document_crawler/OUT/AV3/CV/")

train.shape
test.shape

train.columns.values

model = Word2Vec.load("SG_300features_10minwords_5context")
type(model.wv.syn0)
model.wv.syn0.shape

from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=True ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5a. Remove extra Stop words
    moreStopwords = ["b","full","value","file","period","months","additional","basis","street","respect","cases","application","amount","since","costs","address","well","days","series","c","particular","purposes","purpose","proceedings","proceeding","h","able","ba","country","held","board","first","second","final","judge","non","statement","documents","many","notes","note","j","considered","aw","echr","whether","language","ill","time","taken","kh","rovd","must","set","within","p","mr","mrs","provided","sher","one","new","route","routes","three","would","previously","shall","en","k","g","applicants","eur","date","might","paragraph","u","kd","could","made","company","see","public","parking","statements","article","government","business","information","therefore","right","also","applicant","court","act","state","security","section","hearing","v","service","case","law","person","courts","regional","january","february","march","april","may","june","july","august","september","october","november","december"]
    more_meaningful_words = [token for token in words if token not in moreStopwords]
    #
    # 5. Return a list of words
    return(more_meaningful_words)
    #return(words)



import nltk.data
#nltk.download()   

import sys  

#reload(sys)  
#sys.setdefaultencoding('utf8')

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=True ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    # raw_sentences = tokenizer.tokenize(review.strip())
    raw_sentences = tokenizer.tokenize(review.strip().decode("utf8"))
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocabulary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec
    


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
     
    for review in reviews:
       # Print a status message every # review
       if counter%100. == 0.:
           print "Review %d of %d" % (counter, len(reviews))
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
       #Increment the counter
       counter = counter + 1
    return reviewFeatureVecs

import time

start = time.time() # Start time

clean_train_reviews = []
for review in train["sentences"]:
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print "Creating average feature vecs for test sentences"
clean_test_reviews = []
for review in test["sentences"]:
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

end = time.time()
elapsed = end - start
print "Time taken for Creating average feature vecs for test sentences: ", elapsed, "seconds."

C:\Users\danny\AppData\Local\Enthought\Canopy\User\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\danny\AppData\Local\Enthought\Canopy\User\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Review 0 of 4000
Review 100 of 4000
Review 200 of 4000
Review 300 of 4000
Review 400 of 4000
Review 500 of 4000
Review 600 of 4000
Review 700 of 4000
Review 800 of 4000
Review 900 of 4000
Review 1000 of 4000
Review 1100 of 4000
Review 1200 of 4000
Review 1300 of 4000
Review 1400 of 4000
Review 1500 of 4000
Review 1600 of 4000
Review 1700 of 4000
Review 1800 of 4000
Review 1900 of 4000
Review 2000 of 4000
Review 2100 of 4000
Review 2200 of 4000
Review 2300 of 4000
Review 2400 of 4000
Review 2500 of 4000
Review 2600 of 4000
Review 2700 of 4000
Review 2800 of 4000
Review 2900 of 4000
Review 3000 of 4000
Review 3100 of 4000
Review 3200 of 4000
Review 3300 of 4000
Review 3400 of 4000
Review 3500 of 4000
Review 3600 of 4000
Review 3700 of 4000
Review 3800 of 4000
Review 3900 of 4000
Creating average feature vecs for test sentences
Review 0 of 2348
Review 100 of 2348
Review 200 of 2348
Review 300 of 2348
Review 400 of 2348
Review 500 of 2348
Review 600 of 2348
Review 700 of 2348
Review 800 of

In [7]:
from tempfile import TemporaryFile
outfile = trainDataVecs

from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.grid_search import GridSearchCV

grid_values = {'C':[30]} # Decide which settings you want for the grid search. 

# Perform classification with SVM, kernel=rbf
classifier_rbf = GridSearchCV(svm.SVC(), 
                        grid_values, 
                        scoring = 'roc_auc', cv = 20) 
classifier_rbf.fit(trainDataVecs, train["AV3"] )
prediction_rbf = classifier_rbf.predict(testDataVecs)

# Perform classification with SVM, kernel=linear
classifier_linear = GridSearchCV(svm.SVC(kernel='linear'), 
                        grid_values, 
                        scoring = 'roc_auc', cv = 20) 
classifier_linear.fit(trainDataVecs, train["AV3"] )
prediction_linear = classifier_linear.predict(testDataVecs)

# Perform classification with SVM, kernel=linear
classifier_liblinear = GridSearchCV(svm.LinearSVC(), 
                        grid_values, 
                        scoring = 'roc_auc', cv = 20) 
classifier_liblinear.fit(trainDataVecs, train["AV3"] )
prediction_liblinear = classifier_liblinear.predict(testDataVecs)



# Print results
print("Results for SVC(kernel=rbf)")
output = pd.DataFrame(data={"title":test["title"],"conclusion":test["conclusion"], "AV3":prediction_rbf})
output.to_csv( "A3009_prediction_rbf.csv", index=False, quoting=3, escapechar='\\' , sep='^')

print("Results for SVC(kernel=linear)")
output = pd.DataFrame(data={"title":test["title"],"conclusion":test["conclusion"], "AV3":prediction_linear})
output.to_csv( "A3009_prediction_linear.csv", index=False, quoting=3, escapechar='\\' , sep='^')

print("Results for LinearSVC()")
output = pd.DataFrame(data={"title":test["title"],"conclusion":test["conclusion"], "AV3":prediction_liblinear})
output.to_csv( "A3009_prediction_liblinear.csv", index=False, quoting=3, escapechar='\\' , sep='^')

output = pd.DataFrame(data={"AV3_1":prediction_rbf, "AV3_2":prediction_linear, "AV3_3":prediction_liblinear})
output.to_csv( "A3009_SVC_Summary.csv", index=False, quoting=3, escapechar='\\' , sep='^')

Results for SVC(kernel=rbf)
Training time: 173.603000s; Prediction time: 4.817000s
Results for SVC(kernel=linear)
Training time: 89.673000s; Prediction time: 2.497000s
Results for LinearSVC()
Training time: 73.025000s; Prediction time: 0.013000s


In [8]:
from sklearn.externals import joblib
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

import time

idx=10000000000000
start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th (for 5 times the vocab size, use 1/25th ) of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0
num_clusters = word_vectors.shape[0] / 25

idx=0
batch_size=45
# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )
#kmeans_clustering = MiniBatchKMeans(n_clusters=num_clusters, random_state=39, init_size=10758)
#kmeans_clustering = MiniBatchKMeans(n_clusters=num_clusters)
#kmeans_clustering = MiniBatchKMeans(init='k-means++', n_clusters=num_clusters, batch_size=batch_size, n_init=10, max_no_improvement=10, verbose=0)
##idx = kmeans_clustering.fit_transform(word_vectors)
#idx = kmeans_clustering.fit_predict(word_vectors)

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."


joblib.dump(kmeans_clustering,  'doc_cluster.pkl')

# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))


Time taken for K Means clustering:  435.789999962 seconds.


In [9]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [10]:
train_centroids = np.zeros( (train["sentences"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["sentences"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1
    
    

In [11]:

#classifier_rbf = svm.SVC()
t0 = time.time()
#classifier_rbf.fit(trainDataVecs, train["AV3"])
classifier_rbf = GridSearchCV(svm.SVC(), 
                        grid_values, 
                        scoring = 'roc_auc', cv = 20) 
classifier_rbf.fit(train_centroids, train["AV3"] )
t1 = time.time()
prediction_rbf = classifier_rbf.predict(test_centroids)
t2 = time.time()
time_rbf_train = t1-t0
time_rbf_predict = t2-t1

# Perform classification with SVM, kernel=linear
#classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
#classifier_linear.fit(trainDataVecs, train["AV3"])
classifier_linear = GridSearchCV(svm.SVC(kernel='linear'), 
                        grid_values, 
                        scoring = 'roc_auc', cv = 20) 
classifier_linear.fit(train_centroids, train["AV3"] )
t1 = time.time()
prediction_linear = classifier_linear.predict(test_centroids)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

# Perform classification with SVM, kernel=linear
#classifier_liblinear = svm.LinearSVC()
t0 = time.time()
#classifier_liblinear.fit(trainDataVecs, train["AV3"])
classifier_liblinear = GridSearchCV(svm.LinearSVC(), 
                        grid_values, 
                        scoring = 'roc_auc', cv = 20) 
classifier_liblinear.fit(train_centroids, train["AV3"] )
t1 = time.time()
prediction_liblinear = classifier_liblinear.predict(test_centroids)
t2 = time.time()
time_liblinear_train = t1-t0
time_liblinear_predict = t2-t1



# Print results in a nice table
print("Results for SVC(kernel=rbf)")
print("Training time: %fs; Prediction time: %fs" % (time_rbf_train, time_rbf_predict))
#print(classification_report(test["name"], prediction_rbf))
output = pd.DataFrame(data={"title":test["title"],"conclusion":test["conclusion"], "AV3":prediction_rbf})
output.to_csv( "A3009_Centroid_prediction_rbf.csv", index=False, quoting=3, escapechar='\\' , sep='^')

print("Results for SVC(kernel=linear)")
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
#print(classification_report(test[1], prediction_linear))
output = pd.DataFrame(data={"title":test["title"],"conclusion":test["conclusion"], "AV3":prediction_linear})
output.to_csv( "A3009_Centroid_prediction_linear.csv", index=False, quoting=3, escapechar='\\' , sep='^')

print("Results for LinearSVC()")
print("Training time: %fs; Prediction time: %fs" % (time_liblinear_train, time_liblinear_predict))
#print(classification_report(test["title"], prediction_liblinear))
output = pd.DataFrame(data={"title":test["title"],"conclusion":test["conclusion"], "AV3":prediction_liblinear})
output.to_csv( "A3009_Centroid_prediction_liblinear.csv", index=False, quoting=3, escapechar='\\' , sep='^')

output = pd.DataFrame(data={"AV3_1":prediction_rbf, "AV3_2":prediction_linear, "AV3_3":prediction_liblinear})
output.to_csv( "A3009_Centroid_SVC_Summary.csv", index=False, quoting=3, escapechar='\\' , sep='^')

Results for SVC(kernel=rbf)
Training time: 885.433000s; Prediction time: 21.096000s
Results for SVC(kernel=linear)
Training time: 395.502000s; Prediction time: 3.431000s
Results for LinearSVC()
Training time: 37.538000s; Prediction time: 0.020000s
